# Analyse de la qualité des données du fichier Clients_Master


## Résumé des anomalies observées
- 500 clients au lieu de 300 000 clients actifs mentionnés dans l'énoncé
- 18 doublons (3%)
- 2 doublons "partiels": les champs sont identiques à l'exception de ville (renseigné /null)
<br> problème d'unicité pour l'ID --> deux sources ont pris le même id mais on fait deux enregistrements au lieu d'une mise à jour de la ville du client
- 17% de villes non renseignées
<br> --> saisie manuelle facultative
- 61% des lignes comportent des villes qui n'existe pas ou son mal orthographiées (Pariss)
<br> --> saisie manuelle (faute de frappe, mauvais champs sélectionné)
- 177 emails invalides
  - Emails = 'invalid_email' : 10 (1.9%)
  - Emails invalides (présence d'accents) : 167 (32.1%) dont 14 avec des espaces

- Les champs nom et prénom ne présentent pas d'anomalie (complétude OK)
- Il n'y a pas d'incohérence entre les dates d'inscription et de dernière activité

### Points positifs
- ✅ **RGPD: conservation des données** pas de client dans la BDD avec activité > 3 ans (+5 ans archives)

## Proposition d'une correction
- supprimer les doublons (pas de perte de données)
<br>--> 18 lignes supprimées
<br> --> 2 lignes avec ID en doublon supprimées
- correction automatique avec Fuzzy matching = 1 353 lignes corrigées
- suppression des noms de villes invalides non corrigeables
<br> --> donnée inexploitable = 103 villes null
- correction des accents sur les emails

## Ce que l'on ne peut pas corriger
- villes non renseignées (non bloquant pour l'analyse)
- email non renseigné (invalid) car boutiques physiques donc email facultatif
<br> --> pas d'email en doublon en dehors de ceux invalid

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import unicodedata
from fuzzywuzzy import process

In [2]:
df_clients = pd.read_csv('../data/raw/Clients_Master.csv')

In [3]:
print("AUDIT DATAFRAME CLIENTS")

display(df_clients.head())
display(df_clients.info())

print("\n---Taux de valeurs manquantes (%)---")
missing_percentage = df_clients.isnull().sum() / len(df_clients) * 100
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

print("\n--- Vérification des doublons ---")
clients_doublons = df_clients.duplicated().sum()
print(f"Nombre total de doublons: {clients_doublons} / soit {clients_doublons/len(df_clients) * 100:.2f}%")

print("\n--- Vérification des doublons sur le client_id ---")
nb_doublons_id = df_clients['client_id'].duplicated().sum()
print(f"Nombre de client_id en doublon : {nb_doublons_id}")

print("\n--- Vérification de complétude : nom et prénom ---")
nb_nom_manquant = df_clients['nom'].isna().sum()
nb_prenom_manquant = df_clients['prenom'].isna().sum()
print(f"Nombre de noms manquants : {nb_nom_manquant}")
print(f"Nombre de prénoms manquants : {nb_prenom_manquant}")

print("\n--- Vérification de cohérence des dates --- ")
df_clients['date_inscription'] = pd.to_datetime(df_clients['date_inscription'], errors='coerce')
df_clients['derniere_activite'] = pd.to_datetime(df_clients['derniere_activite'], errors='coerce')

mask_incoherence = df_clients['date_inscription'] > df_clients['derniere_activite']

nb_incoherences = mask_incoherence.sum()
pct_incoherences = (nb_incoherences / len(df_clients)) * 100

print(f"Incohérences détectées : {nb_incoherences} ({pct_incoherences:.2f}%)")

AUDIT DATAFRAME CLIENTS


,client_id,nom,prenom,email,ville,date_inscription,derniere_activite
0,1,Rey,Alexandrie,alexandrie.rey@live.com,Lejeune,2023-10-15,2025-01-21
1,2,Lagarde,Eugène,eugène.lagarde@yahoo.fr,NaN,2024-07-31,2025-08-25
2,3,Gaudin,Aimée,aimée.gaudin@sfr.fr,Lopezdan,2024-01-03,2025-10-19
3,4,Faivre,Noël,noël.faivre@voila.fr,Saint Eugène,2023-08-01,2025-07-11
4,5,Faure,Véronique,véronique.faure@club-internet.fr,Sainte Jacques-les-Bains,2023-02-14,2025-10-23


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   client_id          520 non-null    int64 
 1   nom                520 non-null    object
 2   prenom             520 non-null    object
 3   email              520 non-null    object
 4   ville              428 non-null    object
 5   date_inscription   520 non-null    object
 6   derniere_activite  520 non-null    object
dtypes: int64(1), object(6)
memory usage: 28.6+ KB


None


---Taux de valeurs manquantes (%)---
ville    17.692308
dtype: float64

--- Vérification des doublons ---
Nombre total de doublons: 18 / soit 3.46%

--- Vérification des doublons sur le client_id ---
Nombre de client_id en doublon : 20

--- Vérification de complétude : nom et prénom ---
Nombre de noms manquants : 0
Nombre de prénoms manquants : 0

--- Vérification de cohérence des dates --- 
Incohérences détectées : 0 (0.00%)


In [4]:
print("\n--- AUDIT DES EMAILS ---")

email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

# Compter spécifiquement "invalid_email"
nb_invalid_email = (df_clients['email'] == 'invalid_email').sum()

# Validation des autres emails
mask_valide = df_clients['email'].apply(
    lambda x: bool(re.match(email_pattern, str(x).strip())) if pd.notna(x) else False
)

nb_valides = mask_valide.sum()
nb_invalides_autres = (~mask_valide & df_clients['email'].notna() & (df_clients['email'] != 'invalid_email')).sum()
nb_nulls = df_clients['email'].isna().sum()

print(f"Emails valides : {nb_valides} ({(nb_valides/len(df_clients)*100):.1f}%)")
print(f"Emails = 'invalid_email' : {nb_invalid_email} ({(nb_invalid_email/len(df_clients)*100):.1f}%)")
print(f"Emails invalides (présence d'accents) : {nb_invalides_autres} ({(nb_invalides_autres/len(df_clients)*100):.1f}%)")
print(f"Emails null : {nb_nulls} ({(nb_nulls/len(df_clients)*100):.1f}%)")

# Afficher des exemples d'emails invalides (hors "invalid_email")
if nb_invalides_autres > 0:
    print("\n Exemples d'emails invalides (hors 'invalid_email') :")
    emails_invalides_autres = df_clients[
        ~mask_valide &
        df_clients['email'].notna() &
        (df_clients['email'] != 'invalid_email')
    ]
    display(emails_invalides_autres[['email']].head(10))


--- AUDIT DES EMAILS ---
Emails valides : 343 (66.0%)
Emails = 'invalid_email' : 10 (1.9%)
Emails invalides (présence d'accents) : 167 (32.1%)
Emails null : 0 (0.0%)

 Exemples d'emails invalides (hors 'invalid_email') :


,email
1,eugène.lagarde@yahoo.fr
2,aimée.gaudin@sfr.fr
3,noël.faivre@voila.fr
4,véronique.faure@club-internet.fr
10,céline.benard@sfr.fr
12,frédérique.fabre@orange.fr
16,charles.dupré@voila.fr
27,émile.lecoq@hotmail.fr
32,théophile.potier@bouygtel.fr
37,pénélope.delahaye@noos.fr


## On vérifie si les villes renseignées existent
## Le script est long (~1min30 pour tester chaque ville)

In [5]:
print("\n--- VALIDATION DES NOMS DE VILLES ---")

def valider_villes_dataframe(df, colonne_ville='ville'):
    villes_uniques = df[colonne_ville].dropna().unique()

    villes_invalides = []
    villes_valides = []

    for ville in villes_uniques:
        url = f"https://geo.api.gouv.fr/communes?nom={ville}&limit=1"
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200 and len(response.json()) > 0:
                villes_valides.append(ville)
            else:
                villes_invalides.append(ville)
        except:
            villes_invalides.append(ville)

    print(f"Villes valides : {len(villes_valides)} soit {len(villes_valides)/len(villes_uniques)*100:.0f}%")
    print(f"Villes invalides : {len(villes_invalides)} soit {len(villes_invalides)/len(villes_uniques)*100:.0f}%")

    if villes_invalides:
        # Afficher les lignes concernées
        mask_invalides = df[colonne_ville].isin(villes_invalides)
        print(f"\nNombre de lignes affectées : {mask_invalides.sum()} soit {mask_invalides.sum()/len(df_clients)*100:.0f}%")
        display(df[mask_invalides][[colonne_ville]].head(10))

    return villes_valides, villes_invalides

villes_valides, villes_invalides = valider_villes_dataframe(df_clients)



--- VALIDATION DES NOMS DE VILLES ---
Villes valides : 87 soit 24%
Villes invalides : 272 soit 76%

Nombre de lignes affectées : 317 soit 61%


,ville
0,Lejeune
2,Lopezdan
4,Sainte Jacques-les-Bains
6,Petit-sur-Mer
14,Sainte Lucieboeuf
15,Augernec
16,Pichon-sur-Mer
17,Blanc-sur-Mer
19,Saint ValentineVille
22,Saint Thomasdan


In [7]:
df_ventes = pd.read_csv('../data/raw/Ventes_Q1_2025.csv')

In [11]:
print("\n--- POLITIQUE DE CONSERVATION RGPD ---")

# Configuration
DUREE_CONSERVATION_STANDARD = 3  # années
DUREE_CONSERVATION_PROSPECT = 3  # années (prospects non-acheteurs)
DUREE_ARCHIVAGE = 5  # années supplémentaires en archive

# Convertir dates en forçant les erreurs à NaT
df_clients['derniere_activite'] = pd.to_datetime(df_clients['derniere_activite'], errors='coerce')
df_clients['date_inscription'] = pd.to_datetime(df_clients['date_inscription'], errors='coerce')
date_reference = pd.to_datetime(df_ventes['date']).max()

# Vérifier s'il y a des dates invalides
nb_dates_invalides = df_clients['derniere_activite'].isna().sum()
if nb_dates_invalides > 0:
    print(f"⚠️ {nb_dates_invalides} dates invalides détectées dans 'derniere_activite'")

# Identifier si le client a déjà acheté
clients_acheteurs = df_ventes['client_id'].unique()
df_clients['a_achete'] = df_clients['client_id'].isin(clients_acheteurs)

# Calculer l'ancienneté (en gérant les NaT)
df_clients['annees_inactivite'] = (date_reference - df_clients['derniere_activite']).dt.days / 365

# Déterminer l'action RGPD
def determiner_action_rgpd(row):
    # Si pas de date d'activité valide, considérer comme à supprimer
    if pd.isna(row['annees_inactivite']):
        return "SUPPRIMER (date invalide)"

    if row['a_achete']:
        if row['annees_inactivite'] > DUREE_CONSERVATION_STANDARD + DUREE_ARCHIVAGE:
            return "SUPPRIMER"
        elif row['annees_inactivite'] > DUREE_CONSERVATION_STANDARD:
            return "ARCHIVER"
        else:
            return "CONSERVER"
    else:  # Prospect
        if row['annees_inactivite'] > DUREE_CONSERVATION_PROSPECT:
            return "SUPPRIMER"
        else:
            return "CONSERVER"

df_clients['action_rgpd'] = df_clients.apply(determiner_action_rgpd, axis=1)

# Statistiques
stats_actions = df_clients['action_rgpd'].value_counts()
total = len(df_clients)

print(f"Date de référence : {date_reference.date()}")
print(f"Politique : {DUREE_CONSERVATION_STANDARD} ans actif + {DUREE_ARCHIVAGE} ans archive\n")

print("--- Répartition des actions RGPD ---")
for action in ["CONSERVER", "ARCHIVER", "SUPPRIMER", "SUPPRIMER (date invalide)"]:
    count = stats_actions.get(action, 0)
    if count > 0:
        pourcentage = (count / total * 100)
        emoji = "✅" if action == "CONSERVER" else "📦" if action == "ARCHIVER" else "🗑️"
        print(f"{emoji} {action:30s}: {count:5d} ({pourcentage:5.1f}%)")


--- POLITIQUE DE CONSERVATION RGPD ---
Date de référence : 2025-10-29
Politique : 3 ans actif + 5 ans archive

--- Répartition des actions RGPD ---
✅ CONSERVER                     :   520 (100.0%)


# Correction du dataframe

## Suppression des doublons

In [6]:
print("\n--- SUPPRESSION DES DOUBLONS ---")

df_clients_without_duplicate = df_clients.drop_duplicates()
print(f"Lignes avant : {len(df_clients)}")
print(f"Lignes après : {len(df_clients_without_duplicate)}")
print(f"Nombre de doublons restants: {df_clients_without_duplicate.duplicated().sum()}")

# suppression spéciale pour les doublons sur l'id (obligation d'unicité)
df_clients_without_duplicate = df_clients_without_duplicate.drop_duplicates(subset=['client_id'], keep='first')

print("\n--- Vérification des doublons sur le client_id ---")
nb_doublons_id = df_clients_without_duplicate['client_id'].duplicated().sum()
print(f"Nombre de client_id en doublon : {nb_doublons_id}")


if nb_doublons_id > 0:
    # Afficher les client_id en doublon
    client_id_dupliques = df_clients_without_duplicate[df_clients_without_duplicate['client_id'].duplicated(keep=False)]
    print(f"\nNombre total de lignes concernées : {len(client_id_dupliques)}")

    # Afficher quelques exemples de doublons
    print("\nExemples de lignes avec client_id en doublon :")
    display(client_id_dupliques.sort_values('client_id').head(10))

print("\n--- Vérification des doublons sur l'email autres que  invalid_email---")
df_emails_valides = df_clients_without_duplicate[df_clients_without_duplicate['email'] != 'invalid_email']
nb_doublons_email = df_emails_valides['email'].duplicated().sum()

print(f"Nombre d'emails en doublon (hors 'invalid_email') : {nb_doublons_email}")

if nb_doublons_email > 0:
    # Afficher les client_email en doublon
    client_email_dupliques = df_clients_without_duplicate[df_clients_without_duplicate['email'].duplicated(keep=False)]
    print(f"\nNombre total de lignes concernées : {len(client_email_dupliques)}")

    # Afficher quelques exemples de doublons
    print("\nExemples de lignes avec email en doublon :")
    display(client_email_dupliques.sort_values('email').head(10))



--- SUPPRESSION DES DOUBLONS ---
Lignes avant : 520
Lignes après : 502
Nombre de doublons restants: 0

--- Vérification des doublons sur le client_id ---
Nombre de client_id en doublon : 0

--- Vérification des doublons sur l'email autres que  invalid_email---
Nombre d'emails en doublon (hors 'invalid_email') : 0


In [7]:
print("\n--- SAUVEGARDE DU FICHIER NETTOYE ---")

# Sauvegarder dans le dossier processed
df_clients_without_duplicate.to_csv('../data/processed/Clients_Master_corrected.csv', index=False)



--- SAUVEGARDE DU FICHIER NETTOYE ---


# Correction des noms de ville par Fuzzy Matching 
### Penser à installer la librairie : pip install fuzzywuzzy python-Levenshtein
### ⚠️ Script long: >7min

In [8]:
# Chargement du DataFrame précédemment nettoyé
df_clients_cleaned = pd.read_csv('../data/processed/Clients_Master_corrected.csv')

# Charger le référentiel des communes
url_communes = "https://www.data.gouv.fr/fr/datasets/r/dbe8a621-a9c4-4bc3-9cae-be1699c5ff25"
communes_france = pd.read_csv(url_communes, sep=',')

# Créer une liste des noms de communes (en conservant la casse originale)
noms_communes_lower = set(communes_france['nom_commune_complet'].str.lower())
noms_communes_original = communes_france['nom_commune_complet'].tolist()

print(f"✅ {len(noms_communes_lower)} communes chargées")

def corriger_ville_automatique(nom_ville, villes_reference):
    """Trouve la ville la plus proche dans le référentiel"""
    if pd.isna(nom_ville):
        return None

    meilleur_match, score = process.extractOne(nom_ville, villes_reference)

    if score > 80:  # Seuil de similarité
        return meilleur_match
    return None

print("\n--- VALIDATION ET CORRECTION DES VILLES ---")

# Identifier les villes invalides
villes_uniques = df_clients_cleaned['ville'].dropna().unique()
villes_a_corriger = {}

for ville in villes_uniques:
    if ville.lower() not in noms_communes_lower:
        # Ville invalide, chercher une correction
        ville_corrigee = corriger_ville_automatique(ville, noms_communes_original)
        if ville_corrigee:
            villes_a_corriger[ville] = ville_corrigee
        else:
            villes_a_corriger[ville] = None


# Appliquer les corrections
if villes_a_corriger:
    print(f"\n--- APPLICATION DES CORRECTIONS ---")

    nb_corrections = 0
    nb_null = 0

    for ville_invalide, ville_corrigee in villes_a_corriger.items():
        if ville_corrigee:
            # Correction automatique réussie
            df_clients_cleaned['ville'] = df_clients_cleaned['ville'].replace(ville_invalide, ville_corrigee)
            nb_corrections += (df_clients_cleaned['ville'] == ville_corrigee).sum()
        else:
            # Aucune correction trouvée → remplacer par null
            mask_invalide = df_clients_cleaned['ville'] == ville_invalide
            nb_null += mask_invalide.sum()
            df_clients_cleaned.loc[mask_invalide, 'ville'] = None

    print(f"✅ {nb_corrections} lignes corrigées")
    print(f"⚠️ {nb_null} lignes avec un nom de ville inexploitable -> remplacé par null")
else:
    print("\n✅ Toutes les villes sont valides, aucune correction nécessaire")

# Afficher le résultat
print(f"\n📋 État final après correction")
print(f"Villes valides : {df_clients_cleaned['ville'].notna().sum()}")
print(f"Villes null : {df_clients_cleaned['ville'].isna().sum()}")

✅ 33610 communes chargées

--- VALIDATION ET CORRECTION DES VILLES ---

--- APPLICATION DES CORRECTIONS ---
✅ 1353 lignes corrigées
⚠️ 13 lignes avec un nom de ville inexploitable -> remplacé par null

📋 État final après correction
Villes valides : 399
Villes null : 101


In [9]:
# Validation de la correction apportée
villes_valides, villes_invalides = valider_villes_dataframe(df_clients_cleaned)

Villes valides : 179 soit 98%
Villes invalides : 4 soit 2%

Nombre de lignes affectées : 4 soit 1%


,ville
105,Le Mesnil-Mauger
129,Évaillé
275,Saint-Gabriel-Brécy
443,St barthelemy


### Ce sont des faux-négatifs, ces quatres villes existent bien.
<br>Il doit y avoir un problème de référencement entre les deux sources utilisées pour la correction
<br> On peut considérer que la correction a réussi à traiter tout ce qui était possible

In [10]:
print("\n--- SAUVEGARDE DU FICHIER AVEC VILLES CORRIGEES ---")

# Sauvegarder dans le dossier processed
df_clients_cleaned.to_csv('../data/processed/Clients_Master_corrected.csv', index=False)


--- SAUVEGARDE DU FICHIER AVEC VILLES CORRIGEES ---


In [11]:
print("\n--- NETTOYAGE DES EMAILS (accents + casse) ---")
df_clients_cleaned = pd.read_csv('../data/processed/Clients_Master_corrected.csv')

def nettoyer_email(email):
    """Retire les accents et met en minuscules"""
    if pd.isna(email):
        return email

    email_str = str(email).strip().lower()  # Minuscules et trim

    # Supprimer tous les espaces
    email_str = email_str.replace(' ', '')

    # Retirer les accents
    email_normalise = unicodedata.normalize('NFD', email_str)
    email_propre = ''.join(
        char for char in email_normalise
        if unicodedata.category(char) != 'Mn'
    )

    return email_propre

# Appliquer le nettoyage
df_clients_cleaned['email'] = df_clients_cleaned['email'].apply(nettoyer_email)

print("\n--- AUDIT DES EMAILS ---")

email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

# Compter spécifiquement "invalid_email"
nb_invalid_email = (df_clients_cleaned['email'] == 'invalid_email').sum()

# Validation des autres emails
mask_valide = df_clients_cleaned['email'].apply(
    lambda x: bool(re.match(email_pattern, str(x).strip())) if pd.notna(x) else False
)

nb_valides = mask_valide.sum()
nb_invalides_autres = (~mask_valide & df_clients_cleaned['email'].notna() & (df_clients_cleaned['email'] != 'invalid_email')).sum()
nb_nulls = df_clients_cleaned['email'].isna().sum()

print(f"Emails valides : {nb_valides} ({(nb_valides/len(df_clients_cleaned)*100):.1f}%)")
print(f"Emails = 'invalid_email' : {nb_invalid_email} ({(nb_invalid_email/len(df_clients_cleaned)*100):.1f}%)")
print(f"Emails invalides (présence d'accents ou espaces) : {nb_invalides_autres} ({(nb_invalides_autres/len(df_clients_cleaned)*100):.1f}%)")
print(f"Emails null : {nb_nulls} ({(nb_nulls/len(df_clients_cleaned)*100):.1f}%)")

# Afficher des exemples d'emails invalides (hors "invalid_email")
if nb_invalides_autres > 0:
    print("\n Exemples d'emails invalides (hors 'invalid_email') :")
    emails_invalides_autres = df_clients_cleaned[
        ~mask_valide &
        df_clients_cleaned['email'].notna() &
        (df_clients_cleaned['email'] != 'invalid_email')
    ]
    display(emails_invalides_autres[['email']].head(10))


--- NETTOYAGE DES EMAILS (accents + casse) ---

--- AUDIT DES EMAILS ---
Emails valides : 490 (98.0%)
Emails = 'invalid_email' : 10 (2.0%)
Emails invalides (présence d'accents ou espaces) : 0 (0.0%)
Emails null : 0 (0.0%)


In [12]:
print("\n--- SAUVEGARDE DU FICHIER AVEC EMAILS CORRIGEES ---")

# Sauvegarder dans le dossier processed
df_clients_cleaned.to_csv('../data/processed/Clients_Master_corrected.csv', index=False)


--- SAUVEGARDE DU FICHIER AVEC EMAILS CORRIGEES ---
